# Exploration

In [1]:
import pandas as pd
import csv
import re
from collections import Counter, defaultdict
from itertools import islice
from tqdm import tqdm_notebook
from fog.key import fingerprint

In [2]:
# Params
DATA = '../data/interventions.csv'
OUTPUT = '../data/didascalies.csv'

# Match for "group"
GROUPS = [
    ('LREM', ('larem', 'lrem', 'rem'), 'en marche'),
    ('MODEM', ('modem', ), 'mouvement démocrate'),
    ('LR', ('lr', ), 'républicains'),
    ('LFI', ('lfi', 'fi'), 'insoumise'),
    ('UAI', ('udi', 'lc', 'uai'), 'constructifs'),
    ('NG', ('ng', ), 'nouvelle gauche'),
    ('GDR', ('gdr', ), 'gauche démocrate'),
    ('NI', ('ni', ), 'non inscrit')
]

GROUPS = [(t, tuple(re.compile('\\b' + p + '\\b') for p in ps), e) for t, ps, e in GROUPS]

BLACK_LIST = [
    'article',
    'motion',
    'séance', # Possibilité d'étudier les suspensions via ce mot-clé
    'seance',
    'amendement',
    'projet',
    'scrutin',
    'proposition',
    'votants',
    'remplace',
    'adopté',
    'tirage au sort'
]

EXCLAMATION_RE = re.compile('«([^»]+)»')
HYPHENS_RE = re.compile('[\\-‐‒–—―−‑⁃]')
QUOTES_RE = re.compile('[«»]')
DIDASCALIES_SPLITTER_RE = re.compile('[.\s]-')

In [3]:
# Helpers
TAGS_RE = re.compile(r'<[^>]+>')

def strip_tags(html):
    return re.sub(TAGS_RE, '', html)

def is_didascalie(line):
    if line['nom'] != 'NULL' or line['parlementaire'] != 'NULL':
        return False
    
    raw_intervention = fingerprint(strip_tags(re.sub(QUOTES_RE, '', line['intervention'])))
    
    if fingerprint(re.sub(QUOTES_RE, '', line['titre'].replace(' (suite)', ''))) == raw_intervention:
        return False
    
    for b in BLACK_LIST:
        if b in raw_intervention:
            print(raw_intervention)
            return False
    
    return True

def split_didascalies(intervention):
    intervention = strip_tags(intervention)
    intervention = re.sub(HYPHENS_RE, '-', intervention)
    
    return [d.strip('. ') for d in DIDASCALIES_SPLITTER_RE.split(intervention.strip('. '))]

def extract_groups(didascalie):
    raw_didascalie = didascalie.lower()
    
    groups = set()
    
    for name, patterns, expression in GROUPS:
        if expression in raw_didascalie:
            groups.add(name)
            
            continue
            
        for pattern in patterns:
            if re.search(pattern, raw_didascalie):
                groups.add(name)
                break
                
    return list(groups)

In [4]:
def find_reference_point(lines, didascalie_line, index):
    index -= 1
    
    while is_didascalie(lines[index]) or (lines[index]['nom'] == 'NULL' and lines[index]['parlementaire'] == 'NULL'):
        index -=1
        
    if lines[index]['seance_id'] != didascalie_line['seance_id']:
        print(didascalie_line)
        print(lines[index])
        print()
        
    return lines[index]

In [5]:
DIDASCALIES = []

with open(DATA, 'r') as f:
    reader = csv.DictReader(f)
    
    LINES = list(reader)
    
    for i, line in enumerate(LINES):
        if not is_didascalie(line):
            continue
            
        didascalies = split_didascalies(line['intervention'])
        
        reference_point = find_reference_point(LINES, line, i)
        
        for didascalie in didascalies:
            meta = dict(reference_point)
            meta['didascalie'] = didascalie

            DIDASCALIES.append(meta)

a est heures la ouverte quinze seance
a dix est heures le ouvert quinze scrutin
a cinq cinquantecinq est heures la reprise seance seize suspendue
a est heures la levee quinze seance seize
compte de directrice du joly la lassemblee nationalecatherine rendu seancede service
a est heures la ouverte quinze seance
a est heures le ouvert quinze scrutin
a dix dixsept est heures la quinze reprise seance suspendue
a dixhuit dixsept est heures la quinze reprise seance suspendue vingt
a dixhuit dixneuf est heures la reprise seance suspendue trentecinq vingtcinq
a dixneuf est heures la quarantecinq reprise seance suspendue trentecinq vingtdeux
2017 29 a cinquantecinq est heure heures jeudi juin la le reprise seance suspendue trente vingtdeux zero
a est heure la levee seance trentecinq zero
compte de directrice du joly la lassemblee nationalecatherine rendu seancede service
a est heures la ouverte quinze seance
a est heures la reprise seance seize suspendue trente trentecinq
a cinq dixhuit est heur

233 accepte adopte est gouvernement lamendement le no par rectifie
5 adopte amende bis est larticle
557 est lamendement no retire
234 accepte adopte est gouvernement lamendement le no par
228 accepte adopte est gouvernement lamendement le no par
5 adopte amende est larticle ter
de et la reprise seance suspension
a cinq dix est heures la onze quarantecinq reprise seance suspendue
au est il procede scrutin
absolue70pour de exprimes139majorite ladoption16contre123 nombre suffrages votants149nombre
179 adopte lamendement nest no pas
67 est lamendement no retire
32 adopte gouvernement lamendement le nest no par pas repousse
246 adopte gouvernement lamendement le nest no par pas repousse
au est il procede scrutin
absolue86pour de exprimes170majorite ladoption64contre106 nombre suffrages votants183nombre
247 adopte lamendement nest no pas
au est il procede scrutin
absolue91pour de exprimes180majorite ladoption111contre69 nombre suffrages votants192nombre
34 adopte est lamendement no
34 adopte

249 417 adoptes amendements et identiques les ne nos pas sont
250 adopte lamendement nest no pas
355 adopte lamendement nest no pas
144 adopte lamendement nest no pas
520 adopte lamendement nest no pas
414 452 adoptes amendements et identiques les ne nos pas sont
183 251 adoptes amendements aux et les mis ne nos pas sont successivement voix
180 204 321 adoptes amendements et identiques lamendement les no nos sont tombe
165 39 73 adoptes amendements aux et les mis ne nos pas sont successivement voix
482 adopte lamendement nest no pas
111 146 40 adoptes amendements et identiques les ne nos pas sont
205 29 378 418 501 517 adoptes amendements aux et les mis ne nos pas sont successivement voix
112 adopte lamendement nest no pas
105 adopte lamendement nest no pas
491 adopte lamendement nest no pas
379 adopte lamendement nest no pas
322 461 adoptes amendements et identiques les ne nos pas rectifie sont
41 adopte lamendement nest no pas
102 adopte lamendement nest no pas
113 325 99 adoptes ame

86 87 adoptes amendements aux et les mis ne nos pas sont successivement voix
65 adopte lamendement nest no pas
73 adopte commission la lamendement nest no par pas repousse
61 adopte lamendement nest no pas
59 adopte commission la lamendement nest no par pas repousse
88 adopte lamendement nest no pas
89 adopte lamendement nest no pas
15 80 adoptes amendements aux et les mis ne nos pas sont successivement voix
99 adopte lamendement nest no pas
69 adopte commission la lamendement nest no par pas repousse
60 adopte lamendement nest no pas
90 adopte lamendement nest no pas
25 adopte commission la lamendement nest no par pas repousse
au est il procede scrutin
absolue56pour de exprimes111majorite ladoption100contre11 nombre suffrages votants115nombre
adopte ainsi de est la larticle lensemble proposition que resolution unique
adopte ainsi de est la larticle lensemble proposition que resolution unique
absolue56pour de exprimes111majorite ladoption100contre11 nombre suffrages votants115nombre
11

a cinquantecinq est heure la levee seance une
compte de directrice du joly la lassemblee nationalecatherine rendu seancede service
a est heures la ouverte seance seize
2018 de finances loi pour projet
2018 de finances loi pour projet
a est heures la ouverte seance seize
163 243 349 adoptes amendements aux et les mis ne nos pas sont successivement voix
au est il procede scrutin
absolue45pour de exprimes88majorite ladoption80contre8 nombre suffrages votants89nombre
27 adopte est larticle
27 adopte est larticle
absolue45pour de exprimes88majorite ladoption80contre8 nombre suffrages votants89nombre
de et la reprise seance suspension
a cinq dixneuf est heures la reprise seance suspendue vingt
1146 1380 adopte consequence en est lamendement no tombe
28 a adoptes amendes et larticle letat sont
a est heures la levee seance vingt vingtcinq
compte de directrice du joly la lassemblee nationalecatherine rendu seancede service
a est heures la ouverte quinze seance
de et la reprise seance suspension

...

2859 adopte lamendement nest no pas
1625 adopte lamendement nest no pas
3191 adopte lamendement nest no pas
1336 adopte lamendement nest no pas
1596 adopte lamendement nest no pas
938 est lamendement no retire
1536 est lamendement no retire
1536 est lamendement no retire
2830 adopte gouvernement lamendement le nest no par pas repousse
au est il procede scrutin
absolue32pour de exprimes62majorite ladoption55contre7 nombre suffrages votants66nombre
35 adopte amende est larticle
1884 est lamendement no rectifie retire
au est il procede scrutin
absolue28pour de exprimes55majorite ladoption12contre43 nombre suffrages votants57nombre
2727 adopte lamendement nest no pas
1400 adopte commission et gouvernement la lamendement le nest no par pas repousse
au est il procede scrutin
absolue27pour de exprimes52majorite ladoption13contre39 nombre suffrages votants61nombre
2705 444 adoptes amendements et identiques les ne nos pas rectifie sont
de et la reprise seance suspension
a dixhuit est heures la 

de et la reprise seance suspension
a est et heures la quarante quarantetrois reprise seance suspendue une vingttrois
220 adopte est lamendement no
11 18 adoptes amendements et identiques les ne nos pas sont
au est il procede scrutin
absolue29pour de exprimes56majorite ladoption9contre47 nombre suffrages votants57nombre
24 6 8 adoptes amendements et identiques les ne nos pas sont
162 adopte gouvernement lamendement le nest no par pas repousse
19 adopte lamendement nest no pas
de et la reprise seance suspension
2018 8 a est heure immediatement juin la le quinze reprise seance suspendue vendredi zero
au est il procede scrutin
absolue26pour de exprimes51majorite ladoption37contre14 nombre suffrages votants54nombre
209 adopte est lamendement no
209 adopte est lamendement no
absolue26pour de exprimes51majorite ladoption37contre14 nombre suffrages votants54nombre
82 adopte lamendement nest no pas
12 adopte lamendement nest no pas
214 accepte adopte est gouvernement lamendement le no par
71 ac

16 a est heures la ouverte seance
au est il procede scrutin
absolue32pour de exprimes63majorite ladoption10contre53 nombre suffrages votants64nombre
adoptee de la motion nest pas prealable rejet
au est il procede scrutin
absolue28pour de exprimes54majorite ladoption19contre35 nombre suffrages votants57nombre
adoptee commission de en la motion nest pas renvoi
a cinquantecinq dixneuf est heures la levee seance
compte de directrice du joly la lassemblee nationalecatherine rendu seancede service
a est et heures la ouverte seance trente une vingt
12 a est heure juin la le levee mardi seance une
compte de directrice du joly la lassemblee nationalecatherine rendu seancede service
a est heures la neuf ouverte seance trente
est immediatement instants la quelques reprise seance suspendue
de et la reprise seance suspension
a de deux dix est heures la m presidence reprise seance sous suspendue sylvain waserman
de et la reprise seance suspension
a cinquante cinquantecinq de dix est heures jego la m

1827 adopte commission et gouvernement la lamendement le nest no par pas repousse
1184 1559 1626 757 adoptes amendements et identiques les ne nos pas sont
1685 244 71 adoptes amendements et identiques les ne nos pas sont
1589 adopte est lamendement no
730 adopte est lamendement no
2191 527 adoptes amendements et identiques les ne nos pas sont
944 accepte adopte commission est et gouvernement la lamendement le no par
945 accepte adopte commission est et gouvernement la lamendement le no par
1731 adopte lamendement nest no pas
1318 217 478 70 712 73 adoptes amendements et gouvernement identiques le les ne nos par pas repousses sont
218 630 adoptes amendements commission et gouvernement identiques la le les ne nos par pas repousses sont
245 adopte commission et gouvernement la lamendement le nest no par pas repousse
2131 accepte adopte commission est et gouvernement la lamendement le no par
632 adopte commission et gouvernement la lamendement le nest no par pas repousse
1319 188 246 74 76

18 adopte lamendement nest no pas
adoptee de est la loi proposition
2018 22 a cinquantecinq est heure juin la le levee seance vendredi zero
compte de directrice du joly la lassemblee nationalecatherine rendu seancede service


In [6]:
Counter(d['didascalie'].lower() for d in DIDASCALIES if 'même' in d['didascalie'].lower()).most_common()

[('mêmes mouvements', 485),
 ('applaudissements sur les mêmes bancs', 14),
 ('mêmes mouvements sur les mêmes bancs', 10),
 ('même mouvement', 8),
 ('nouveaux applaudissements sur les mêmes bancs', 3),
 ('exclamations persistantes sur les mêmes bancs', 3),
 ('même mouvements', 3),
 ('exclamations sur les mêmes bancs', 2),
 ('protestations sur les mêmes bancs', 2),
 ('mêmes mouvement', 2),
 ('« eh oui ! » sur les mêmes bancs', 1),
 ('« si ! » sur les mêmes bancs', 1),
 ('nouvelles protestations sur les mêmes bancs', 1),
 ('« non ! » sur les mêmes bancs', 1),
 ('brouhaha grandissant sur les mêmes bancs', 1),
 ('nouvelles exclamations sur les mêmes bancs', 1),
 ('mêmes mouvements et exclamations sur les bancs des groupes rem et modem',
  1),
 ('« ah ! » sur les mêmes bancs', 1),
 ('vifs applaudissements sur les mêmes bancs', 1),
 ('« bravo ! » et rires sur les mêmes bancs', 1),
 ('les mêmes se lèvent et applaudissent longuement', 1),
 ('« pas de la même manière ! »sur les bancs du groupe l

In [7]:
# Handling "mêmes mouvements"
MEMES_MOUVEMENTS_RE = re.compile('mêmes?\s+mouvements?')

for i, d in enumerate(DIDASCALIES):
    if re.search(MEMES_MOUVEMENTS_RE, d['didascalie'].lower()):
        # print(d['didascalie'])
        last_d = DIDASCALIES[i - 1]
        meta = dict(last_d)
        meta['didascalie'] = MEMES_MOUVEMENTS_RE.sub(d['didascalie'], last_d['didascalie'])
        DIDASCALIES[i] = meta

In [8]:
# Extracting groups
for d in DIDASCALIES:
    groups = extract_groups(d['didascalie'])
    d['groups'] = '|'.join(groups)

In [9]:
# All groups at once
ALL_GROUPS_KEY = '|'.join(g for g, _, _ in GROUPS)
for d in DIDASCALIES:
    raw_didascalie = d['didascalie'].lower()
        
    if 'tous les banc' in raw_didascalie or 'mmes et mm. les députés' in raw_didascalie:
        d['groups'] = ALL_GROUPS_KEY

In [10]:
# Handling "mêmes bancs"
MEMES_BANCS_RE = re.compile('mêmes?\s+bancs?')

for i, d in enumerate(DIDASCALIES):
    raw_didascalie = d['didascalie'].lower()
    
    if re.search(MEMES_BANCS_RE, raw_didascalie):
        last_d = DIDASCALIES[i - 1]
        
        groups = set(d['groups'].split('|') if d['groups'] else []) | set(last_d['groups'].split('|') if last_d['groups'] else [])

        d['groups'] = '|'.join(groups)

In [11]:
pd.DataFrame(DIDASCALIES).head(1000)

,date,didascalie,fonction,groups,id,intervention,moment,nb_mots,nom,parlementaire,parlementaire_groupe_acronyme,seance_id,sexe,source,timestamp,titre,titre_complet,type
0,2017-06-27,Applaudissements,"président, doyen d'âge",,14,<p>Monsieur le secrétaire d'État chargé des re...,15:00,636,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,170,allocution du doyen d'âge,allocution du doyen d'âge,loi
1,2017-06-27,Il est procédé au tirage au sort,"président, doyen d'âge",,17,"<p>L'ordre du jour appelle, conformément à l'a...",15:00,140,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,220,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
2,2017-06-27,Il est procédé au tirage au sort,"président, doyen d'âge",,19,<p>Sont désignés scrutateurs titulaires : M. B...,15:00,51,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,250,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
3,2017-06-27,Mmes et MM. les députés et M. le secrétaire d'...,"président, doyen d'âge",LREM|MODEM|LR|LFI|UAI|NG|GDR|NI,26,<p>La séance est reprise.</p><p>Mes chers coll...,15:00,63,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,350,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
4,2017-06-27,Applaudissements,"président, doyen d'âge",,28,<p>M. Jean-Charles Taugourdeau : 94 voix</p>,15:00,6,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,370,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
5,2017-06-27,Applaudissements,"président, doyen d'âge",,30,<p>Mme Laure de La Raudière : 34 voix</p>,15:00,9,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,390,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
6,2017-06-27,Applaudissements,"président, doyen d'âge",,32,<p>Mme Laurence Dumont : 32 voix</p>,15:00,6,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,410,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
7,2017-06-27,Applaudissements,"président, doyen d'âge",,34,<p>Mme Caroline Fiat : 30 voix</p>,15:00,6,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,430,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
8,2017-06-27,Mmes et MM. les députés et M. le secrétaire d'...,"président, doyen d'âge",LREM|MODEM|LR|LFI|UAI|NG|GDR|NI,36,<p>M. François de Rugy ayant obtenu la majorit...,15:00,34,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,450,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi
9,2017-06-27,M. François de Rugy prend place au fauteuil pr...,"président, doyen d'âge",,36,<p>M. François de Rugy ayant obtenu la majorit...,15:00,34,NULL,Bernard Brochand,NI,1,H,http://www.assemblee-nationale.fr/15/cri/2016-...,450,Élection du président de l'assemblée nationale,Élection du président de l'assemblée nationale,loi


In [12]:
Counter(d['parlementaire'] if d['parlementaire'] != 'NULL' else d['nom'] for d in DIDASCALIES if 'applaud' in d['didascalie'].lower()).most_common()

[('Edouard Philippe', 367),
 ('Gérard Collomb', 312),
 ('Bruno le Maire', 294),
 ('Gérald Darmanin', 289),
 ('Agnès Buzyn', 233),
 ('Jean-Luc Mélenchon', 225),
 ('Éric Coquerel', 212),
 ('Muriel Pénicaud', 204),
 ('Danièle Obono', 185),
 ('François de Rugy', 164),
 ('Nicole Belloubet', 161),
 ('André Chassaigne', 159),
 ('Sébastien Jumel', 155),
 ('François Ruffin', 153),
 ('Alexis Corbière', 136),
 ('Julien Denormandie', 129),
 ('Stéphane Travert', 128),
 ('Jean-Michel Blanquer', 126),
 ('Jacques Mézard', 125),
 ('Ugo Bernalicis', 118),
 ('Stéphane Peu', 117),
 ('Bastien Lachaud', 112),
 ('Marc Fesneau', 110),
 ("Loïc Prud'homme", 110),
 ('Nicolas Hulot', 107),
 ('Gilles Lurton', 106),
 ('Fabien Di Filippo', 105),
 ('Christian Jacob', 103),
 ('Jacqueline Gourault', 94),
 ('Mathilde Panot', 93),
 ('élisabeth Borne', 93),
 ('Richard Ferrand', 91),
 ('Fabien Roussel', 91),
 ('Adrien Quatennens', 91),
 ('Christophe Castaner', 91),
 ('Jean-Paul Lecoq', 90),
 ('Clémentine Autain', 90),
 ('V

In [13]:
Counter(d['parlementaire_groupe_acronyme'] for d in DIDASCALIES if 'applaud' in d['didascalie'].lower()).most_common()

[('LREM', 3649),
 ('NULL', 3451),
 ('LR', 2335),
 ('LFI', 1821),
 ('GDR', 1039),
 ('MODEM', 958),
 ('NG', 887),
 ('UAI', 491),
 ('LC', 300),
 ('NI', 281)]

In [14]:
Counter(d['sexe'] for d in DIDASCALIES if 'applaud' in d['didascalie'].lower()).most_common()

[('H', 8109), ('F', 3652), ('NULL', 3451)]

In [15]:
Counter(d['didascalie'] for d in DIDASCALIES if d['parlementaire'] == 'Jean Lassalle').most_common()

[('Sourires', 34),
 ('Rires', 13),
 ('Applaudissements sur divers bancs', 3),
 ('Applaudissements sur les bancs des groupes FI et GDR', 2),
 ('Applaudissements sur les bancs du groupe LR', 2),
 ('rires sur plusieurs bancs', 1),
 ('Applaudissements et rires sur plusieurs bancs', 1),
 ('Applaudissements sur les bancs des groupes REM et MODEM, ainsi que sur plusieurs bancs de chacun des autres groupes',
  1),
 ('Applaudissements sur plusieurs bancs de chacun des groupes', 1),
 ('Sourires sur plusieurs bancs', 1),
 ('Exclamations', 1),
 ('Plusieurs députés du groupe LR se lèvent et applaudissent', 1),
 ('Applaudissements sur les bancs du groupe REM et sur quelques bancs du groupe FI',
  1),
 ('Applaudissements sur plusieurs bancs du groupe REM', 1),
 ('Murmures', 1),
 ('Rires et applaudissements sur de nombreux bancs', 1),
 ('Applaudissements sur les bancs du groupe LR et sur quelques bancs du groupe REM',
  1),
 ('Applaudissements sur les bancs du groupe FI', 1),
 ('Rires et applaudisseme

In [16]:
for d in DIDASCALIES:
    raw_didascalie = d['didascalie'].lower()
    
    if 'pupitr' in raw_didascalie or 'claquem' in raw_didascalie:
        print(d['didascalie'])

Claquements de pupitre sur un banc du groupe LR
Claquements de pupitres sur plusieurs bancs des groupes REM et MODEM
Exclamations et claquements de pupitres sur les bancs du groupe REM
Claquements de pupitre sur quelques bancs du groupe REM
Claquements de pupitres sur les bancs du groupe REM
Exclamations et claquements de pupitre sur les bancs du groupe REM
Très vives protestations et claquements de pupitres sur les bancs des groupes LR, UDI-Agir, NG et GDR
Très vives protestations et claquements de pupitres sur les bancs des groupes LR, UDI-Agir, NG et GDR
Claquements de pupitres sur les bancs du groupe LaREM
Exclamations et claquements de pupitres sur les bancs du groupe LR
Claquements de pupitres sur certains bancs des députés non inscrits
Exclamations sur les bancs des groupes FI et NG, claquements de pupitre sur les bancs du groupe GDR
Claquements de pupitres sur les bancs du groupe GDR, puis des groupes LR, FI et NG
Les claquements de pupitres s'intensifient
Les claquements de pu

In [17]:
Counter(d['parlementaire'] for d in DIDASCALIES if 'rappel' in d['didascalie'].lower() and 'présid' not in d['fonction']).most_common()

[('NULL', 48),
 ('Sébastien Jumel', 12),
 ('Sébastien Chenu', 5),
 ('Jean-Paul Lecoq', 4),
 ('André Chassaigne', 4),
 ('Naïma Moutchou', 4),
 ('Marc Le Fur', 4),
 ('Olivier Véran', 4),
 ('Stéphane Peu', 4),
 ('Richard Lioger', 4),
 ('Gérard Cherpion', 3),
 ('Olivier Faure', 3),
 ('Danièle Obono', 3),
 ('Éric Coquerel', 3),
 ('Christian Hutin', 3),
 ('Philippe Latombe', 2),
 ('Nadia Hai', 2),
 ('Valérie Boyer', 2),
 ('Ludovic Pajot', 2),
 ('Fabien Di Filippo', 2),
 ('Pierre Dharréville', 2),
 ('Valérie Rabault', 2),
 ('Julien Borowczyk', 2),
 ('Thierry Benoit', 2),
 ('Emmanuelle Ménard', 2),
 ('Ugo Bernalicis', 2),
 ('Pierre-Henri Dumont', 2),
 ('Thibault Bazin', 2),
 ('Delphine Batho', 2),
 ('Christelle Dubos', 2),
 ('François Pupponi', 2),
 ('Mickaël Nogal', 1),
 ('David Habib', 1),
 ('Philippe Gosselin', 1),
 ('Christian Jacob', 1),
 ('Laurent Saint-Martin', 1),
 ('Véronique Louwagie', 1),
 ('Fabien Roussel', 1),
 ('Joël Giraud', 1),
 ('Alexis Corbière', 1),
 ('Valérie Lacroute', 1),

In [18]:
Counter(d['didascalie'].lower() for d in DIDASCALIES if 'rappel' in d['didascalie'].lower()).most_common()

[('rappel au règlement', 257),
 ('rappels au règlement', 64),
 ('« rappel au règlement ! » sur les bancs des groupes lr, ng et fi', 1),
 ("« ce n'est pas un rappel au règlement ! » sur divers bancs", 1)]

In [19]:
# Normalizing `parlementaire_groupe_acronyme`
for d in DIDASCALIES:
    g = d['parlementaire_groupe_acronyme']
    g = 'UAI' if g == 'LC' else g
    
    d['parlementaire_groupe_norme'] = g

In [29]:
# Adding tonality marker


POSITIVE_KEY_WORDS = [
  'applaud',
  'sourir',
  'oui',
  'approb',
  'approuv',
  'rir',
  'assentiment'
];

NEGATIVE_KEY_WORDS = [
  'protest',
  'non',
  'exclam',
  'rappel au règlement',
  'rappels au règlement',
  'interrupt',
  'scandal',
  'oh',
  'dénégat',
  'faux',
  'stop',
  'hué',
  'brouhaha',
  'sifflement',
  'tumult'
];

for d in DIDASCALIES:
  didasc = d['didascalie']
  ton = None
  for word in POSITIVE_KEY_WORDS:
        if (word in didasc):
            ton = 'positif'
            break
            
  for word in NEGATIVE_KEY_WORDS:
        if (word in didasc):
            ton = 'negatif'
            break

  if (ton is None):
    ton = 'neutre'
  d['ton'] = ton;

In [30]:
with open(OUTPUT, 'w') as f:
    writer = csv.DictWriter(f, fieldnames=list(DIDASCALIES[0].keys()))
    writer.writeheader()
    
    for d in DIDASCALIES:
        writer.writerow(d)
print('written at', OUTPUT)

written at ../data/didascalies.csv


In [31]:
# Sanity test
for d in DIDASCALIES:
    if d['nom'] == 'NULL' and d['parlementaire'] == 'NULL':
        print(d['intervention'])

In [32]:
sum(int(d['nom'] == 'NULL' and d['parlementaire'] == 'NULL') for d in DIDASCALIES)

0